# Globals

In [127]:
#@ markdown Run this cell to load the data. Double click this text to load a custom dataset.

#########################################
# Option 1: Load data from Google sheet #
#########################################

"""
The spreadsheet needs three subsheets, named "Votes", "Chart", "Titles"
Reference sheet: https://docs.google.com/spreadsheets/d/12QQ6aC2SsDjtlT7u5kcbFiKBAQWU0JA_rq2CEKciwTk/edit?usp=sharing
"""
SHEETNAME = '2000s Movies'

default_delimiter = ". " # Example vote: 1. [Film123]
special_delimiters = ["\) "] # Provide alterantive delimiters here, e.g. 1) [Film123]

##############################################################
# Option 2: Paste the vote matrix directly here as a string. #
##############################################################

vote_matrix = ""

if vote_matrix != "": load_from_sheet = True

# Imports

In [164]:
import math
import numpy as np
import pandas as pd
pd.set_option('max_rows', None)

if load_from_sheet:

  from google.colab import drive
  drive.mount('/content/drive')

  from google.colab import auth
  auth.authenticate_user()

  import gspread
  from gspread_dataframe import set_with_dataframe
  from oauth2client.client import GoogleCredentials
  gc = gspread.authorize(GoogleCredentials.get_application_default())

  votesheet = gc.open(SHEETNAME).worksheet('Votes')
  chartsheet = gc.open(SHEETNAME).worksheet('Chart')
  titlessheet = gc.open(SHEETNAME).worksheet('Titles')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
def get_votes_df(votesheet):

  votes = np.array(votesheet.get_all_values())
  votes_df = pd.DataFrame(votes[1:,:], columns=votes[0], index=range(1,len(votes[:,0])))

  # Tag unranked votes
  for voter in votes_df:
    votes_df[voter] = votes_df[voter].mask(votes_df[voter].str.startswith("["), "-1. " + votes_df[voter].astype(str))
    
    #remove comments after ]
    #votes_df[voter] = votes_df[voter].str.split("]", expand=True) + "]"
    votes_df[voter] = votes_df[voter].str.split("]", n=1, expand=True) + "]"

  # Replace non-default delimiters
  for sd in special_delimiters:
    votes_df.replace(sd, default_delimiter, regex=True, inplace=True)

  return votes_df.mask(votes_df=="]", "0")

votes_df = get_votes_df(votesheet)
votes_df.head()

,aflickering,AinsleyIscariot,Allyn,amanda_online,Aruji,astadaradim,avalyn2,Bach,Blazko,BeepisBoopis,BlueSkies,bobohead61x,carlitos_go,Celebros,Cicadidae,comrade_sue,connordenney,crushedtoapulp,Cubbierill,darsu,DeathOfSeasons,Dew1400,diction,Dolwphin,DtzaraD,dukkha,Faure,goodsinginggum,Havey,hollowchatter,Inovox,Iron_Dwarf,Jabbi,Jag114,JessieEye,JoLeKosovo,Josh318,jpzitoleopold,KnightToMourning,LifeDuringWartime,LowFidelity,Magenta_Bob,Meister333,MicrophoneFiend,momotaroyouth,moorezy,mrblond,mistercb,nadenrhys,nostalghia,nervalitis,NeverTrump,nivekwriter,novocaine69,Ohg,oscarmurraey,Ovals,peelsa,Rehtek,Rengar18,rillo_raffy,russalex86,RuthJanelle,sesherhesher,Silverblack,she_esh,simo000,Sluggo714,spencership,Stoicorum,Strawhenge,SynysterShadows,tectactoe,thefeverayeaye,tonkatonka,tuukkis,YasashiiDia
1,1. [Film1870],1. [Film1870],01. [Film27],01. [Film50],1. [Film2721],1. [Film50],1. [Film1441],1. [Film178],1. [Film2328],1. [Film3613],-1. [Film50],1. [Film1441],1. [Film333],1. [Film2635],1. [Film16833],1. [Film639],01. [Film7747],01. [Film50],1. [Film178],1. [Film4262],1. [Film49540],1. [Film227],1. [Film1164],1. [Film19683],1. [Film2338],1. [Film18655],01. [Film288],1. [Film1870],1. [Film18655],01. [Film1491],1. [Film639],1. [Film1441],1. [Film93],01. [Film1511],1. [Film1164],1. [Film2721],1. [Film1208],1. [Film678],1. [Film180],1. [Film19083],1. [Film812],1. [Film3871],-1. [Film16956],1. [Film639],1. [Film8],1. [Film68],1. [Film1386],01. [Film205],1. [Film1491],1. [Film678],1. [Film6376],01. [Film768],1. [Film470],1. [Film782],1. [Film18655],1. [Film19683],1. [Film18971],1. [Film19683],1. [Film2721],1. [Film1001],1. [Film68],1. [Film2338],1. [Film68],1. [Film50],1. [Film50],1. [Film7747],1. [Film49540],1. [Film639],1. [Film68],1. [Film1870],1. [Film59],-1. [Film150],1. [Film6208],1. [Film288],1. [Film4859],1. [Film639],1. [Film639]
2,2. [Film276],2. [Film227],02. [Film1334],02. [Film23474],2. [Film180],2. [Film8061],2. [Film1164],2. [Film532],2. [Film27],2. [Film1035],-1. [Film288],2. [Film639],2. [Film7679],2. [Film1223],2. [Film1223],2. [Film10718],02. [Film8040],02. [Film1164],2. [Film55416],2. [Film1164],2. [Film55292],2. [Film1223],2. [Film4913],2. [Film1234],2. [Film671],2. [Film1939],02. [Film678],2. [Film1374],2. [Film1234],02. [Film18655],2. [Film205],2. [Film91],2. [Film2721],02. [Film11787],2. [Film1441],2. [Film683],2. [Film455],2. [Film4117],2. [Film255],2. [Film7186],2. [Film327],2. [Film288],-1. [Film19683],2. [Film98],2. [Film227],2. [Film1441],2. [Film554],02. [Film207],2. [Film5529],2. [Film6374],2. [Film24262],02. [Film2338],2. [Film107],2. [Film153],2. [Film288],2. [Film18655],2. [Film678],2. [Film18655],2. [Film2574],2. [Film288],2. [Film1164],2. [Film288],2. [Film470],2. [Film343],2. [Film639],2. [Film68],2. [Film639],2. [Film632],2. [Film710],2. [Film68],2. [Film68],-1. [Film338],2. [Film371],2. [Film1208],2. [Film8034],2. [Film1431],2. [Film288]
3,3. [Film2721],3. [Film1223],03. [Film2721],03. [Film726],3. [Film768],3. [Film288],3. [Film98],3. [Film5331],3. [Film1193],3. [Film1230],-1. [Film2929],3. [Film1164],3. [Film11785],3. [Film27375],3. [Film768],3. [Film302],03. [Film59635],03. [Film3473],3. [Film288],3. [Film1562],3. [Film1223],3. [Film327],3. [Film325],3. [Film1441],3. [Film1449],3. [Film27375],03. [Film639],3. [Film6461],3. [Film2642],03. [Film50],3. [Film13597],3. [Film1857],3. [Film11789],03. [Film3745],3. [Film6461],3. [Film38416],3. [Film68],3. [Film2338],3. [Film512],3. [Film20912],3. [Film8094],3. [Film1114],-1. [Film57591],3. [Film280],3. [Film1449],3. [Film2125],3. [Film632],03. [Film1223],3. [Film1219],3. [Film643],3. [Film1001],03. [Film57],3. [Film989],3. [Film330],3. [Film1858],3. [Film1870],3. [Film726],3. [Film768],3. [Film227],3. [Film639],3. [Film2338],3. [Film1562],3. [Film20912],3. [Film1001],3. [Film1164],3. [Film1164],3. [Film2721],3. [Film1441],3. [Film50],3. [Film276],3. [Film113],-1. [Film1062],3. [Film1355],3. [Film15214],3. [Film2338],3. [Film114294],3. [Film1556]
4,4. 

In [166]:
def get_vote_matrix(votes_df):

  vote_matrix = pd.DataFrame()

  for voter in votes_df:
    for i, vote in enumerate(votes_df[voter]):
      if vote == "": break
      try: rank, title = vote.split(". ", 1)
      except ValueError: 
        rank, title = i+1, vote.split(". ", 1)[-1]
      title = title.lstrip()
      vote_matrix.loc[title, voter] = rank

  vote_matrix.fillna(0,inplace=True)
  vote_matrix = vote_matrix.astype(pd.SparseDtype("int", 0))
  vote_matrix = vote_matrix.drop(["0"])
  #print('Density:', vote_matrix.sparse.density, '\nvote_matrix.shape', vote_matrix.shape)
  return vote_matrix

vote_matrix = get_vote_matrix(votes_df)
display(vote_matrix.head())
vote_matrix.shape

,aflickering,AinsleyIscariot,Allyn,amanda_online,Aruji,astadaradim,avalyn2,Bach,Blazko,BeepisBoopis,BlueSkies,bobohead61x,carlitos_go,Celebros,Cicadidae,comrade_sue,connordenney,crushedtoapulp,Cubbierill,darsu,DeathOfSeasons,Dew1400,diction,Dolwphin,DtzaraD,dukkha,Faure,goodsinginggum,Havey,hollowchatter,Inovox,Iron_Dwarf,Jabbi,Jag114,JessieEye,JoLeKosovo,Josh318,jpzitoleopold,KnightToMourning,LifeDuringWartime,LowFidelity,Magenta_Bob,Meister333,MicrophoneFiend,momotaroyouth,moorezy,mrblond,mistercb,nadenrhys,nostalghia,nervalitis,NeverTrump,nivekwriter,novocaine69,Ohg,oscarmurraey,Ovals,peelsa,Rehtek,Rengar18,rillo_raffy,russalex86,RuthJanelle,sesherhesher,Silverblack,she_esh,simo000,Sluggo714,spencership,Stoicorum,Strawhenge,SynysterShadows,tectactoe,thefeverayeaye,tonkatonka,tuukkis,YasashiiDia
[Film1870],1,1,0,0,44,21,0,0,0,0,0,0,0,0,22,0,0,46,4,12,0,35,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,16,0,7,0,0,0,0,0,0,16,0,0,3,0,0,0,0,9,21,0,0,18,45,0,0,0,1,0,0,0,21,0,0,0
[Film276],2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,3,0,0,32,0,0,0,0
[Film2721],3,0,3,0,1,0,0,0,0,0,-1,41,0,31,10,0,0,0,27,36,16,0,0,0,22,38,0,0,0,0,0,0,2,0,0,1,0,37,11,0,0,0,-1,15,29,0,0,28,0,0,0,0,0,5,0,9,0,0,1,0,0,0,0,12,4,0,3,0,0,0,0,-1,0,6,0,0,7
[Film18092],4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
[Film288],5,8,5,0,4,3,6,11,0,0,-1,42,5,0,8,34,4,9,3,16,0,4,4,37,8,18,1,5,0,8,0,0,0,0,5,0,0,6,20,0,5,2,-1,8,17,24,0,7,0,8,0,34,0,0,2,16,41,0,6,2,12,2,9,9,12,23,8,0,5,5,46,0,7,1,5,0,2


(1027, 77)

In [168]:
def get_titles_df(titlessheet):

  titles_arr = np.array(titlessheet.get_all_values())
  titles_df = pd.DataFrame(titles_arr[:,1], index=titles_arr[:,0], columns=["Title"])
  return titles_df

titles_df = get_titles_df(titlessheet)

if len(titles_df) != len(vote_matrix):
  print(len(titles_df),len(vote_matrix))
  print(set(vote_matrix.index) - set(titles_df.index))
  print(set(titles_df.index) - set(vote_matrix.index))
  for title in vote_matrix.index:
    print(title)
  raise Exception("Update titles_df")

def get_vote_matrix_titled(vote_matrix, titles_df):
  vote_matrix = vote_matrix.sort_index()
  titles_df = titles_df.sort_index()
  assert np.sum(vote_matrix.index != titles_df.index) == 0

  get_vote_matrix_titled=pd.DataFrame(vote_matrix.values, columns=vote_matrix.columns, index=[vote_matrix.index,titles_df["Title"]])
  get_vote_matrix_titled.index.names = ["ID", "Title"]
  return get_vote_matrix_titled

vote_matrix = get_vote_matrix_titled(vote_matrix, titles_df)
vote_matrix.head()

,,aflickering,AinsleyIscariot,Allyn,amanda_online,Aruji,astadaradim,avalyn2,Bach,Blazko,BeepisBoopis,BlueSkies,bobohead61x,carlitos_go,Celebros,Cicadidae,comrade_sue,connordenney,crushedtoapulp,Cubbierill,darsu,DeathOfSeasons,Dew1400,diction,Dolwphin,DtzaraD,dukkha,Faure,goodsinginggum,Havey,hollowchatter,Inovox,Iron_Dwarf,Jabbi,Jag114,JessieEye,JoLeKosovo,Josh318,jpzitoleopold,KnightToMourning,LifeDuringWartime,LowFidelity,Magenta_Bob,Meister333,MicrophoneFiend,momotaroyouth,moorezy,mrblond,mistercb,nadenrhys,nostalghia,nervalitis,NeverTrump,nivekwriter,novocaine69,Ohg,oscarmurraey,Ovals,peelsa,Rehtek,Rengar18,rillo_raffy,russalex86,RuthJanelle,sesherhesher,Silverblack,she_esh,simo000,Sluggo714,spencership,Stoicorum,Strawhenge,SynysterShadows,tectactoe,thefeverayeaye,tonkatonka,tuukkis,YasashiiDia
ID,Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Psychedelic Death Vomit],Psychedelic Death Vomit],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The Road Becomes What You Leave],The Road Becomes What You Leave],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
[Film10000],Container,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
[Film10001],Speed Racer,18,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
[Film1001],回路 [Pulse],21,0,0,9,0,0,0,0,0,0,-1,0,0,0,0,0,22,0,0,0,0,0,0,0,31,48,10,0,0,27,0,0,0,0,0,0,0,5,8,0,0,0,0,0,0,0,0,0,0,48,3,0,0,0,0,0,4,0,0,1,0,0,0,3,0,37,0,0,0,0,0,0,0,0,0,0,0


In [169]:
def get_ranked_votes_counts(vote_matrix):

  # Number of ranked votes
  ranked_votes = [max(max(vote_matrix[vote_matrix[col].notna()][col]),0) for col in vote_matrix.columns]

  # Number of unranked votes
  unranked_votes = [vote_matrix[vote_matrix[col]==-1][col].count() for col in vote_matrix.columns]

  # Total votes
  from operator import add
  total_votes= list(map(add, ranked_votes, unranked_votes))

  vm_index = ['ranked_votes','unranked_votes', 'total_votes']
  vm_data = [ranked_votes, unranked_votes, total_votes]

  return pd.DataFrame(vm_data, columns=vote_matrix.columns, index=vm_index)

vmdf = get_ranked_votes_counts(vote_matrix)
vmdf.head()

,aflickering,AinsleyIscariot,Allyn,amanda_online,Aruji,astadaradim,avalyn2,Bach,Blazko,BeepisBoopis,BlueSkies,bobohead61x,carlitos_go,Celebros,Cicadidae,comrade_sue,connordenney,crushedtoapulp,Cubbierill,darsu,DeathOfSeasons,Dew1400,diction,Dolwphin,DtzaraD,dukkha,Faure,goodsinginggum,Havey,hollowchatter,Inovox,Iron_Dwarf,Jabbi,Jag114,JessieEye,JoLeKosovo,Josh318,jpzitoleopold,KnightToMourning,LifeDuringWartime,LowFidelity,Magenta_Bob,Meister333,MicrophoneFiend,momotaroyouth,moorezy,mrblond,mistercb,nadenrhys,nostalghia,nervalitis,NeverTrump,nivekwriter,novocaine69,Ohg,oscarmurraey,Ovals,peelsa,Rehtek,Rengar18,rillo_raffy,russalex86,RuthJanelle,sesherhesher,Silverblack,she_esh,simo000,Sluggo714,spencership,Stoicorum,Strawhenge,SynysterShadows,tectactoe,thefeverayeaye,tonkatonka,tuukkis,YasashiiDia
ranked_votes,50,50,50,20,50,50,50,50,50,21,0,50,50,50,50,50,50,50,50,50,50,35,50,40,50,50,50,50,20,50,5,20,33,50,50,15,50,50,20,50,10,27,0,50,50,50,50,50,10,50,30,50,50,50,10,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,0,50,50,50,50,50
unranked_votes,0,0,0,0,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0,0,0,0,0
total_votes,50,50,50,20,50,50,50,50,50,21,50,50,50,50,50,50,50,50,50,50,50,35,50,40,50,50,50,50,20,50,5,20,33,50,50,15,50,50,20,50,10,27,50,50,50,50,50,50,10,50,30,50,50,50,10,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50


In [170]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def superellipse(x, n=2, a=1, b=1, size=1):
  return b * (size**n - np.abs(x/a)**n)**(1/n)

def linear_pop_multiplier(counts, most_votes, pop_weight):

  theta = np.linspace(-1/most_votes, 1/most_votes, 201)[pop_weight+100]
  b = (1-theta*most_votes)/2
  multipliers = theta * counts + b
  return 2*multipliers

def exp_pop_multiplier(counts, most_votes, pop_weight):

  if pop_weight == 0: return np.ones(len(counts))

  multipliers = 1 + most_votes * np.exp(-(counts-1)**2 / (2*(pop_weight*most_votes)**2))
  multipliers /= 1 + most_votes
  return multipliers

def elliptical_pop_multiplier(counts, most_votes, pop_weight):

  if pop_weight >= 0:
    counts = counts + 2 * (most_votes//2 - counts) + 2
  n = np.linspace(1, 0.1, 101)[np.abs(pop_weight)]
  multipliers = superellipse(counts-1, n=n, a=1, b=1/most_votes, size=most_votes)
  return 2*multipliers

def get_results_df(vote_matrix, Weight, PopWeight, pop_multiplier, MAX_LENGTH=50):

  vote_matrix = vote_matrix.mask(vote_matrix==-1, (MAX_LENGTH+1)/2)

  results = pd.DataFrame(index=vote_matrix.index)
  results["Votes"] = vote_matrix.astype(bool).sum(axis=1)
  MOST_VOTES = max(results["Votes"])

  #score_matrix = vote_matrix.mask(vote_matrix>0,MAX_LENGTH-vote_matrix+1) # Borda
  #score_matrix = vote_matrix.mask(vote_matrix>0, (MAX_LENGTH-1)*np.exp(-(vote_matrix-1)**2 / (2*Weight**2))) # Exponential
  score_matrix = vote_matrix.mask(vote_matrix>0, superellipse(vote_matrix-1,n=Weight,a=1,b=1,size=MAX_LENGTH)) # Elliptical
  results["Score"] = score_matrix.sum(axis=1)
  results["Score"] *= pop_multiplier(results["Votes"], MOST_VOTES, PopWeight)
  results["Score"] = results["Score"].round(1)
  results["Score"] += 0.00001*results["Votes"] # hacky way of breaking ties by number of votes AND use method="min" for tied votes
  results["Rank"] = results["Score"].rank(ascending=False,method='min').astype(int)
  results["Score"] = results["Score"].round(1)

  return results
  
  
def get_chart_df(vote_matrix):

  # Count votes per title
  counts = vote_matrix.astype(bool).sum(axis=1)
  cdf = pd.DataFrame(index=vote_matrix.index)
  cdf['Votes'] = counts.values
  most_votes = max(cdf['Votes'])
  max_length = np.max(vote_matrix.values)

  # Borda count
  results = get_results_df(vote_matrix, Weight=1, PopWeight=0, pop_multiplier=linear_pop_multiplier)

  # Unqiue score
  results["Unique\nScore"] = results["Score"].where(results["Votes"]==1,0)
  results["Unique\nRank"] = results["Unique\nScore"].rank(ascending=False,method='first').astype(int)

  # Popular score
  results['Popular\nScore'] = results['Score']*results['Votes']
  results["Popular\nRank"] = results["Popular\nScore"].rank(ascending=False,method='first').astype(int)

  # Gold medals
  gold_medals = get_results_df(vote_matrix, Weight=0.1, PopWeight=0, pop_multiplier=linear_pop_multiplier)
  gold_medals["Score"] /= max_length
  gold_medals.drop("Votes",axis=1, inplace=True)
  gold_medals.rename({"Score":"Gold\nMedals", "Rank":"Gold\nRank"}, axis=1, inplace=True)

  # Esoteric score
  esoteric_results = get_results_df(vote_matrix, Weight=0.4, PopWeight=-50, pop_multiplier=elliptical_pop_multiplier)
  esoteric_results.drop("Votes",axis=1, inplace=True)
  esoteric_results.rename({"Score":"Esoteric\nScore", "Rank":"Esoteric\nRank"}, axis=1, inplace=True)

  return pd.concat([results, esoteric_results, gold_medals],axis=1)

cdf = get_chart_df(vote_matrix)
cdf.sort_values(by="Gold\nRank").head(10)

,,Votes,Score,Rank,Unique\nScore,Unique\nRank,Popular\nScore,Popular\nRank,Esoteric\nScore,Esoteric\nRank,Gold\nMedals,Gold\nRank
ID,Title,,,,,,,,,,,
[Film639],千と千尋の神隠し [Spirited Away],37,1235.0,4,0.0,952,45695.0,3,64.2,59,6.0,1
[Film50],Inland Empire,29,815.0,10,0.0,906,23635.0,9,81.5,27,5.0,2
[Film68],There Will Be Blood,34,1249.0,3,0.0,970,42466.0,4,72.4,40,4.0,3
[Film1870],"Synecdoche, New York",21,701.5,14,0.0,686,14731.5,16,125.4,5,4.0,4
[Film1441],花樣年華 [In the Mood for Love],36,1295.0,2,0.0,619,46620.0,2,56.8,67,3.0,5
[Film2721],愛のむきだし [Love Exposure],29,1004.5,8,0.0,767,29130.5,8,85.8,20,3.0,6
[Film18655],铁西区 [Tie Xi Qu: West of the Tracks],15,517.5,28,0.0,684,7762.5,34,165.8,2,3.0,7
[Film19683],As I Was Moving Ahead Occasionally I Saw Brief Glimpses of Beauty,14,534.5,27,0.0,696,7483.0,37,159.6,3,3.0,8
[Film288],Mulholland Dr.,55,2157.0,1,0.0,783,118635.0,1,0.3,861,2.0,9


#Push to Sheet

In [171]:
cdf["RYM ID"] = cdf.index.get_level_values(level="ID")
cdf = cdf.sort_values(by="RYM ID")
cdf["Title"] = cdf.index.get_level_values(level="Title")
cols = ["Rank", "Title", "RYM ID", "Votes",	"Score",	"Esoteric\nRank",	"Esoteric\nScore", "Gold\nRank", "Gold\nMedals", "Popular\nRank", "Popular\nScore", "Unique\nRank", "Unique\nScore"]
cdf = cdf[cols]
set_with_dataframe(chartsheet, cdf.sort_values("Rank"), include_index=False)

In [ ]:
cdf.sort_values(by="Esoteric\nRank").head(20)

# Interactive Chart

__Top Weight:__ Determines the distribution of points for each item in a ranked list from a voter. Negative weights give greater emphasis to the items ranked at the top of the list. For a poll with a maximum list size of N, the top ranked item will always get N points.

__Pop Weight:__ Determines the popularity multiplier by which the raw score is multiplied to get the final score. Negative popularity weights emphasize items with few votes, positive weights emphasize items with many votes.

Some example charts:

__Borda Count__: Top Weight = Pop Weight = 0. The default chart. The top item gets N points, the second item gets N-1 points, etc.

__Esoteric Chart__: Top weight = -5, Pop Weight = -150. Highlights items with few voters but high placements.

__Unique Items__: Top Weight = 0, Pop Weight = -200. Ranking of items that have received only one vote.

__Gold Medals__: Top Weight = -10, Pop Weight = 0. Only the top-ranked items get any points. Dividing the score by N yields the number of "gold medals" received by the respective item.

In [198]:
#@markdown

import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display

def hover(hover_color='silver'):
    return dict(selector="tr:hover", props=[("background-color", "%s" % hover_color)])

def alternate_row_colors(background_color='gainsboro'):
  return dict(selector='tr:nth-child(even)',props=[("background-color", "%s" % background_color)])

styles = [hover()]

def style_df(a):
    a_styled = a.style.set_properties(**{'text-align': 'center'})#.hide_index()
    a_styled = a_styled.format("{:.1f}",subset=['Score'])
    # #styles = [alternate_row_colors(),hover()]   
    #styles = [hover()] 
    #a_styled.set_table_styles(styles)

    return a_styled

def filt(vote_matrix, Results, Weight, PopWeight, Plot):

    if np.abs(PopWeight) <= 100:
      pop_multiplier = linear_pop_multiplier
    else:
      PopWeight -= np.sign(PopWeight)*100
      pop_multiplier = elliptical_pop_multiplier

    w_dist = list(np.linspace(0.1,0.9,10)) + list(np.linspace(1,5,11))
    Weight = w_dist[Weight+10]

    results = get_results_df(vote_matrix, Weight, PopWeight, pop_multiplier)

    if Plot:

      fig, ax = plt.subplots(1,2,figsize=(15,5))

      # Points
      x = np.arange(1,51)
      #y = MAX_LENGTH*gaussian(x, 0, Weight)
      y = superellipse(x-1,n=Weight,a=1,b=1,size=MAX_LENGTH)
      ax[0].scatter(x, y)
      ax[0].plot([1,MAX_LENGTH],[MAX_LENGTH,1], label="Borda", ls="--",c="darkgrey")
      ax[0].set_xlabel("List Rank")
      ax[0].set_ylabel("Points")
      ax[0].set_ylim(0,1.1*MAX_LENGTH)

      # Popularity Weighting
      x = np.arange(1,MOST_VOTES+1)
      y = pop_multiplier(x, MOST_VOTES, PopWeight)
      ax[1].scatter(x,y)
      ax[1].set_xlabel("Film Votes")
      ax[1].set_ylabel("Popularity Multiplier")
      ax[1].set_ylim(0,2.1)
      ax[1].axvline(MOST_VOTES,ls="--",c="darkgrey",label=f"Most Votes: {MOST_VOTES_TITLE} ({MOST_VOTES})")
      #ax[1].text(0.5, 0.89, , transform=ax[1].transAxes, fontsize=14, va='top',ha="center")
      
      for a in ax: a.legend(loc="upper center")
      plt.show()

    return results[["Rank","Score","Votes"]].sort_values(by="Rank")[:Results]

def display_df(Results,Weight,PopWeight,Plot,Display):

    a = filt(vote_matrix, Results, Weight,PopWeight, Plot)

    if Display == "DataFrame":
      a_styled = style_df(a)
      #print('Entries:', len(a))
      display(a_styled)

    elif Display == "RYM Print":
      for i in a.index:
        print(f"[b]{a.loc[i,'Rank']:.0f}.[/b] {i[0]} | Score: {a.loc[i,'Score']:.1f} | Votes: {a.loc[i,'Votes']:.0f}")

MAX_LENGTH = np.max(vote_matrix.values) # Maximum list length
COUNTS = vote_matrix.astype(bool).sum(axis=1)
MOST_VOTES_TITLE = COUNTS.sort_values().index[-1][1]
MOST_VOTES = max(COUNTS) # Number of votes of most voted entry

# Widgets
layout={'width': '350px'}
res_w = widgets.IntSlider(min=10, max=len(vote_matrix), step=1,layout=layout,value=20,description='Results:')
pop_weight = widgets.IntSlider(min=-200, max=200, step=1,layout=layout,value=0,description='Pop Weight:')
top_weight = widgets.IntSlider(min=-10, max=10, step=1,layout=layout,value=0,description='Top Weight:')
plot_w = widgets.Checkbox(value=True,description='Plot Point Distribution')
display_w = widgets.Dropdown(options=['DataFrame','RYM Print'], value='DataFrame', description='Display:', disabled=False)

ws = [res_w,plot_w,display_w,display_w,top_weight,pop_weight]

out = widgets.interactive_output(display_df,{'Results':res_w,'Weight':top_weight,'PopWeight':pop_weight,"Plot":plot_w,"Display":display_w})
ui = widgets.GridBox(ws, layout=widgets.Layout(grid_template_columns="repeat(2, 400px)"))

display(ui, out)

GridBox(children=(IntSlider(value=20, description='Results:', layout=Layout(width='350px'), max=1027, min=10),…

Output()